# Demo of Monte Carlo Simulations for Apen, Sampen, and CGR-Renyi entropy measures

In [1]:
import sys, pathlib
print(sys.path)
sys.path.append(r'C:\Users\Wuestney\Documents\GitHub')
print(sys.path)
import datetime
import functools
import json
import numpy as np
from numpy.random import PCG64
import pandas as pd
from mc_measures.gen_mc_transition import get_model, gen_sample
from mc_measures import mc_entropy
import pyusm
import discreteMSE

['C:\\Users\\Wuestney\\Documents\\GitHub\\monte_carlo_sim', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\python39.zip', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\DLLs', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev', '', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib\\site-packages', 'c:\\users\\wuestney\\documents\\github\\pyusm', 'c:\\users\\wuestney\\documents\\github\\discretemse', 'c:\\users\\wuestney\\documents\\github\\markov_chain_simulation', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib\\site-packages\\win32', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Wuestney\\miniconda3\\envs\\monte-carlo-dev\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Wuestney\\.ipython']
['C:\\Users\\Wuestney\\Documents\\

## Define vector of kernel variances as global variable to be used by all functions

In [18]:
# this is same default sig2 array in pyusm.usm_entropy.
sig2v = ('1.000000e-10', '1.778279e-10', '3.162278e-10', '5.623413e-10',
                 '1.000000e-09', '1.778279e-09', '3.162278e-09', '5.623413e-09',
                 '1.000000e-08', '1.778279e-08', '3.162278e-08', '5.623413e-08',
                 '1.000000e-07', '1.778279e-07', '3.162278e-07', '5.623413e-07',
                 '1.000000e-06', '1.778279e-06', '3.162278e-06', '5.623413e-06',
                ' 1.000000e-05', '1.778279e-05', '3.162278e-05', '5.623413e-05',
                '1.000000e-04', '1.778279e-04', '3.162278e-04', '5.623413e-04',
                '1.000000e-03', '1.778279e-03', '3.162278e-03', '5.623413e-03',
                '1.000000e-02', '1.778279e-02', '3.162278e-02', '5.623413e-02',
                '1.000000e-01', '1.778279e-01', '3.162278e-01', '5.623413e-01',
                '1', '1.778279e+00', '3.162278e+00', '5.623413e+00', '10', '1.778279e+01',
                '3.162278e+01', '5.623413e+01', '100')

## Define functions to calculate theta values for different distributions
These definitions are at the top of main.py and called during main.run() to compute expected entropy values for each sample.

In [21]:
def theta_iiduniform(a, sig2v=sig2v):
    # a is the cardinality of the alphabet of the generating function
    #apen and sampen and renyi are expected to give identical results for iid uniformly distributed random numbers
    apen = np.log(a)
    sampen = np.log(a)
    #placeholder for formula
    renyi_disc = np.log(k)
    renyi_cont = {}
    for sig2 in sig2v:
        rn2 = a * ((1/(-12*sig2)) - np.log(2*np.sqrt(sig2)*np.sqrt(np.pi)))
        renyi_cont[sig2] = rn2
    return apen, sampen, renyi_disc, renyi_cont

def theta_markov(MC_model):
    #apen of a Markov chain is equivalent to the entropy rate of the Markov chain
    apen = mc_entropy.markov_apen(MC_model)
    sampen = mc_entropy.markov_sampen(MC_model)
    renyi = None
    return apen, sampen, renyi  

In [6]:
def cgr_renyi(data, sig2v=sig2v, A=None, refseq=None, Plot=False):
    """
    Combines making the usm coordinates from a data sequence and calculating the renyi entropy values on the CGR object.
    
    Parameters
    ----------
    data : STRING OR ARRAY-TYPE OF INTEGERS
        DISCRETE-VALUED DATA SEQUENCE.
    sig2v : VECTOR WITH VARIANCES, SIG2, TO USE WITH PARZEN METHOD
    A : LIST CONTAINING ALL POSSIBLE SYMBOLS OF THE ALPHABET OF THE SEQUENCE USED BY USM CLASS. The default is None.
        If default, will take alphabet as set of unique characters in seq.
    refseq : STRING, optional
        NAME OF SEQUENCE. The default is None.
    Plot : BOOL, optional
        PLOT OPTION PASSED TO renyi2usm().
        OPTION TO PLOT ENTROPY VALUES AS A FUNCTION OF THE LOG OF THE KERNEL
        VARIANCES, SIG2V. ENTROPY VALUES ON THE Y AXIS AND LN(SIG2) VALUES
        ON THE X AXIS.
        
    Returns
    -------
    Dictionary containing renyi quadratic entropy of the USM for each sig2 value.

    """
    cgr = pyusm.USM.make_usm(data, A=A)
    cgr_coords = np.asarray(cgr.fw)
    renyi = pyusm.usm_entropy.renyi2usm(cgr_coords, sig2v, refseq=refseq, Plot=Plot, deep_copy=False)
    return renyi

## Define functions to handle coordinating the RNG instances and use them to generate reproducible random number samples.
These functions are found in "sim_utils/random_samples.py" and imported into main.

In [3]:
def genRandseq(statespace, nobs=100, generator='default', seed=None):
    """
    Generates a random sequence with uniform probability distribution
    from the state space given.

    Parameters
    ----------
    statespace : {INT, STR, ARRAY-LIKE}
        IF statespace IS INTEGER, THIS IS TAKEN TO BE THE SIZE OF THE 
        ALPHABET OF THE STATE SPACE OF THE RANDOM VARIABLE. IF A STRING OR ARRAY-LIKE
        OBJECT, statespace IS TAKEN TO BE THE SET OF DISCRETE-VALUES
        COMPRISING THE STATE SPACE OF THE RANDOM VARIABLE.
    N : INT, DEFAULT=100
        THE LENGTH OF THE RANDOM SEQUENCE TO BE GENERATED. DEFAULT IS 100 SYMBOLS.
    generator : INSTANCE OF numpy.random.Generator CLASS
        DEFAULT IS 'default' WHICH INIDICATES TO USE THE NUMPY default_rng()
        BITGENERATOR WHICH IMPLEMENTS THE CURRENT NUMPY DEFAULT BITGENERATOR.
    seed : {NONE, INT}, OPTIONAL
        SEED TO USE TO FEED THE BITGENERATOR FOR THE RANDOM NUMBER GENERATOR.
        DEFAULT IS NONE.
        
    Returns
    -------
    List of the randomly generated integers.
    
    (deprecated)
    states, seq : ARRAY, ARRAY
        RETURNS ARRAY OF THE STATE VALUES AND ARRAY OF THE INDICES OF THE STATES ARRAY
        THAT CAN BE USED TO CREATE THE RANDOM SEQUENCE OF STATES.
        
        IF THE STATES ARRAY IS JUST A SERIES OF SEQUENTIAL INTEGERS FROM [0:a] THEN THE
        SEQ ARRAY IS THE RANDOM SEQUENCE OF STATES. 
    """

    if generator == 'default':
        # Set seed sequence using value given to seed arg.
        # If value is an entropy value from a previous seed sequence, ss will be identical to
        # the previous seed sequence.
        ss = np.random.SeedSequence(seed)
        # save entropy so that seed sequence can be reproduced later
        rng = np.random.default_rng(ss)
    else:
        #expects generator to be an instance of the np.random.BitGenerator class
        rng = np.random.default_rng(generator)

    if type(statespace) is int:
        a = statespace
        states = np.array([i for i in range(a)])
    elif type(statespace) is str:
        a = len(statespace)
        states = np.array(list(statespace))
    elif type(statespace) is np.ndarray or tuple or list:
        a = len(statespace)
        states = np.array(statespace)
    randints = rng.integers(a, size=nobs)
    seq = states[randints]
    return seq.tolist()

In [3]:
statespace = ['a', 'b', 'c', 'd']
r1 = genRandseq(statespace, 20)
r1

['c',
 'b',
 'a',
 'a',
 'a',
 'a',
 'b',
 'a',
 'd',
 'b',
 'c',
 'a',
 'a',
 'c',
 'd',
 'a',
 'b',
 'b',
 'd',
 'b']

The below Simulator class is designed to implement best practices in Monte Carlo simulation as outlined in {cite}`Morris2019`. These are primarily that samples generated from the same generating function (ie state space) should be generated sequentially so that the states of the random number generator do not overlap across samples. This ensures true independence between the random samples generated.  

Using the Simulator decorator we seed the random number generating function once when the function is first decorated. Then, each time the decorated function is called it will produce N number of samples (defined by nsim) sequentially so that they are completely independent. The seed states are stored for each sample and can be accessed from the attribute 

In [4]:
#define a decorator class to decorate random sample generating functions
#so that they use the same RNG in sequence for all their samples.
class Simulator:
    def __init__(self, func, nsim, seed=None):
        # update wrapper so that the metadata of the returned function is that of func, not the wrapper
        functools.update_wrapper(self, func)
        # func is function to generate random sample from an RNG
        self.func = func
        # nsim is number of random samples to generate
        self.nsim = nsim
        # initiate SeedSequence from seed arg that makes a high-quality seed to initiate the RNG
        # this is the recommended best practice for reproducible results
        # https://numpy.org/doc/stable/reference/random/bit_generators/generated/numpy.random.SeedSequence.html
        self.ss = np.random.SeedSequence(seed)
        self.seed = self.ss.entropy
        # initiate a random number generator using the PCG-64 bitgenerator
        self.rng = np.random.Generator(PCG64(self.ss))
        # empty dict to contain the bitgenerator states at each iteration
        self.bgstateseq = {'initial state' : self.rng.bit_generator.state}
        
    def __call__(self, *args, **kwargs):
        # define process to run when function decorated by Simulator is called
        # *args and **kwargs to be passed to the sample generating func stored in self.func
        #print('inside __call__')
        #print(self.rng.bit_generator.state)
        self.sample = []
        #print('inside wrapped simulate function')
        # set the 'generator' kwarg to be the RNG defined during __init__()
        kwargs['generator'] = self.rng
        for rep in range(self.nsim):
            self.bgstateseq[rep] = self.rng.bit_generator.state
            seq = self.func(*args, **kwargs)
            self.sample.append(seq)
        self.bgstateseq['end'] = self.rng.bit_generator.state
        return self.sample
        #return wrapper

## Generic routine for generating series of random samples

In [7]:
#set the simulator parameters
nsim = 5
seed = None
#set disttype. Options: 'markov', 'uniform', 'regular'
disttype = 'uniform'
if disttype == 'uniform':
    func = genRandseq
elif disttype == 'markov':
    func = gen_sample
#initiate simulator to handle nsim sequential simulations of the random sample func using the same RNG bit generator
sim = Simulator(func, nsim, seed)

In [8]:
#set sample parameters
#set size of alphabet of discrete-valued random variable
a = 4
#set of sample sizes (sequence lengths) to generate during each iteration
nobs = [50, 100]
#name of distribution function. Either "iiduni" if disttype is 'uniform', or f'markovA{a}Order{mc_order}' if disttype is "markov"
distname = "" 
#markov order, determined by disttype
mc_order = 0
#markov transition matrix object, placeholder
MC_model =  None
#set of values making up the discrete-valued state space of the random variable X
states = [chr(ord('a')+i) for i in range(a)]

In [18]:
#create a dict of the true estimand values
if disttype == 'uniform':
    distname = 'iiduni'
    thetas = dict(list(zip(('apen', 'sampen', 'renyi'), theta_iiduniform(a))))
elif disttype == 'markov':
    distname == ''
    thetas = dict(list(zip(('apen', 'sampen', 'renyi'), theta_markov(MC_model))))
#create dict to keep a log of bitgenerator state sequences for each sample
simulatorstates = {}
#empty dict to hold simulated datasets
simulated = {}
estimates = []
for n in nobs:
    if disttype == 'uniform':
        samples = sim(states, n)
    elif disttype == 'markov':
        #add alphabet size, a, to n as the first a states will be dropped from the sample
        T = n + a
        samples = sim(MC_model, states, n)
    sampname = f'{distname}A{a}N{n}'
    simulatorstates[sampname] = sim.bgstateseq
    simulated[sampname] = samples
    values = []
    mvals = [1, 2, 3, 4]
    sig2v = np.genfromtxt('sig2.csv', delimiter=',')
    for i in range(len(samples)):
        vals = {'sample' : i}
        #print(samples[i])
        renyis = cgr_renyi(samples[i], sig2v, A=states, refseq=f'{sampname}i{i}')
        mests = [] 
        for m in mvals:
            est = {'m' : m, 'apen' : apen(samples[i], m)[0], 'sampen' : sampen(samples[i], m, refseq=f'{sampname}i{i}')[0]}
            mests.append(est)
        vals.update({'renyi_hats' : [renyis,], 'theta_hats' : mests})
        values.append(vals)
    estimates.append({'sampname': sampname, 'nobs' : n, 'values' : values})

The sequence iiduniA4N50i0 is unique, there were no m+1-length matches.
Undefined 2 0
The sequence iiduniA4N50i3 is unique, there were no m+1-length matches.
Undefined 6 0
The sequence iiduniA4N50i4 is unique, there were no m+1-length matches.
Undefined 6 0
The sequence iiduniA4N50i5 is unique, there were no m+1-length matches.
Undefined 4 0
The sequence iiduniA4N50i7 is unique, there were no m+1-length matches.
Undefined 6 0
The sequence iiduniA4N50i8 is unique, there were no m+1-length matches.
Undefined 4 0
The sequence iiduniA4N50i12 is unique, there were no m+1-length matches.
Undefined 2 0
The sequence iiduniA4N50i16 is unique, there were no m+1-length matches.
Undefined 4 0
The sequence iiduniA4N50i17 is unique, there were no m+1-length matches.
Undefined 4 0
The sequence iiduniA4N50i18 is unique, there were no m+1-length matches.
Undefined 2 0


In [ ]:
estimates

In [19]:
#save simulated datasets as a json file
#make list to contain the extra args to feed to sim_files.sim_data_dump() 
#in the order [alphabet, data generating distribution, Markov order]
addinfo = [a, distname, mc_order]
outpath = sim_files.create_output_file_path(out_name=f'{distname}A{a}Nsim{nsim}_{datetime.date.today().isoformat()}.json', overide=True)
sim_files.sim_data_dump(simulated, simulatorstates, outpath, *addinfo)
estsoutpath = sim_files.create_output_file_path(out_name=f'{distname}A{a}Nsim{nsim}_{datetime.date.today().isoformat()}_estimates.json', overide=True)
sim_files.sim_est_dump(thetas, estimates, estsoutpath, *addinfo)

The above process is performed by main.run() which takes the following arguments:


In [22]:
fhand = open(estsoutpath)
estimates_loaded= json.load(fhand)
fhand.close()
estimates_loaded

{'Thetas': {'apen': 1.3862943611198906,
  'sampen': 1.3862943611198906,
  'renyi': None},
 'Estimates': [{'sampname': 'iiduniA4N50',
   'nobs': 50,
   'values': [{'sample': 0,
     'renyi_hats': [{'1e-10': -37.077630360514185,
       '1.7783e-10': -35.92631465697923,
       '3.1623e-10': -34.77503113861995,
       '5.6234e-10': -33.62375743414609,
       '1e-09': -32.4724601745261,
       '1.7783e-09': -31.32114447099114,
       '3.1623e-09': -30.16986095263186,
       '5.6234e-09': -29.018587248158003,
       '1e-08': -27.867289988538005,
       '1.7783e-08': -26.715974285003046,
       '3.1623e-08': -25.56469076664377,
       '5.6234e-08': -24.413417062169913,
       '1e-07': -23.26211980254991,
       '1.7783e-07': -22.110804099014956,
       '3.1623e-07': -20.95952058065568,
       '5.6234e-07': -19.80824687618182,
       '1e-06': -18.65694961656182,
       '1.7783e-06': -17.505633913026866,
       '3.1623e-06': -16.354350394667588,
       '5.6234e-06': -15.203076690193729,
       

## Read estimates dataset into a Pandas Dataframe
Functions defined for this process are found in analyze_results.py. 

In [ ]:
def monteCarloSE(biasarray):
    nsim = len(biasarray)
    squaredsum=np.square(biasarray).sum()
    SE = np.sqrt(squaredsum*(1/(nsim*(nsim-1))))

In [ ]:
data = estimates
data = estimates_loaded

In [26]:
theta_hats = pd.json_normalize(data=estimates, record_path=['values', 'theta_hats'],
                               meta=["sampname", "nobs", ["values", "sample"]])
theta_hats.set_index(['sampname', 'nobs', 'values.sample'])
theta_hats

,m,apen,sampen,sampname,nobs,values.sample
0,1,1.239238,1.300478,iiduniA4N50,50,0
1,2,0.823626,1.415282,iiduniA4N50,50,0
2,3,0.365080,2.772589,iiduniA4N50,50,0
3,4,0.030137,Undefined,iiduniA4N50,50,0
4,1,1.189974,1.186474,iiduniA4N50,50,1
...,...,...,...,...,...,...
155,4,0.115525,1.098612,iiduniA4N100,100,18
156,1,1.347985,1.433574,iiduniA4N100,100,19
157,2,1.097899,1.382717,iiduniA4N100,100,19
158,3,0.566553,1.58045,iiduniA4N100,100,19


In [25]:
theta_hatsload = pd.json_normalize(data=estimates_loaded, record_path=['Estimates', 'values', 'theta_hats'],
                               meta=[['Estimates', "sampname"], ['Estimates', "nobs"], ['Estimates', "values", "sample"]])
theta_hatsload

,m,apen,sampen,Estimates.sampname,Estimates.nobs,Estimates.values.sample
0,1,1.239238,1.300478,iiduniA4N50,50,0
1,2,0.823626,1.415282,iiduniA4N50,50,0
2,3,0.365080,2.772589,iiduniA4N50,50,0
3,4,0.030137,Undefined,iiduniA4N50,50,0
4,1,1.189974,1.186474,iiduniA4N50,50,1
...,...,...,...,...,...,...
155,4,0.115525,1.098612,iiduniA4N100,100,18
156,1,1.347985,1.433574,iiduniA4N100,100,19
157,2,1.097899,1.382717,iiduniA4N100,100,19
158,3,0.566553,1.58045,iiduniA4N100,100,19


In [28]:
renyi_hats = pd.json_normalize(data=estimates, record_path=['values', 'renyi_hats'],
                               meta=["sampname", "nobs", ['values', 'sample']])
renyi_hats

,1e-10,1.7783e-10,3.1623e-10,5.6234e-10,1e-09,1.7783e-09,3.1623e-09,5.6234e-09,1e-08,1.7783e-08,...,3.1623,5.6234,10.0,17.783,31.623,56.234,100.0,sampname,nobs,values.sample
0,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.401031,8.536464,9.678797,10.825054,11.973487,13.123155,14.273550,iiduniA4N50,50,0
1,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.396841,8.534096,9.677461,10.824301,11.973063,13.122917,14.273416,iiduniA4N50,50,1
2,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.395543,8.533347,9.677034,10.824059,11.972926,13.122840,14.273372,iiduniA4N50,50,2
3,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.399866,8.535801,9.678422,10.824842,11.973367,13.123088,14.273512,iiduniA4N50,50,3
4,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.400578,8.536205,9.678650,10.824970,11.973440,13.123129,14.273535,iiduniA4N50,50,4
5,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.396603,8.533949,9.677375,10.824251,11.973035,13.122901,14.273407,iiduniA4N50,50,5
6,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.400824,8.536336,9.678721,10.825010,11.973462,13.123141,14.273542,iiduniA4N50,50,6
7,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.404860,8.538626,9.680015,10.825740,11.973873,13.123373,14.273672,iiduniA4N50,50,7
8,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.397117,8.534240,9.677539,10.824344,11.973087,13.122930,14.273423,iiduniA4N50,50,8
9,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.401795,8.536894,9.679038,10.825189,11.973563,13.123198,14.273574,iiduniA4N50,50,9


In [27]:
renyi_hatsload = pd.json_normalize(data=estimates_loaded, record_path=['Estimates', 'values', 'renyi_hats'],
                               meta=[['Estimates', "sampname"], ['Estimates', "nobs"], ['Estimates', "values", "sample"]])
renyi_hatsload

,1e-10,1.7783e-10,3.1623e-10,5.6234e-10,1e-09,1.7783e-09,3.1623e-09,5.6234e-09,1e-08,1.7783e-08,...,3.1623,5.6234,10.0,17.783,31.623,56.234,100.0,Estimates.sampname,Estimates.nobs,Estimates.values.sample
0,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.401031,8.536464,9.678797,10.825054,11.973487,13.123155,14.273550,iiduniA4N50,50,0
1,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.396841,8.534096,9.677461,10.824301,11.973063,13.122917,14.273416,iiduniA4N50,50,1
2,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.395543,8.533347,9.677034,10.824059,11.972926,13.122840,14.273372,iiduniA4N50,50,2
3,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.399866,8.535801,9.678422,10.824842,11.973367,13.123088,14.273512,iiduniA4N50,50,3
4,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.400578,8.536205,9.678650,10.824970,11.973440,13.123129,14.273535,iiduniA4N50,50,4
5,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.396603,8.533949,9.677375,10.824251,11.973035,13.122901,14.273407,iiduniA4N50,50,5
6,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.400824,8.536336,9.678721,10.825010,11.973462,13.123141,14.273542,iiduniA4N50,50,6
7,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.404860,8.538626,9.680015,10.825740,11.973873,13.123373,14.273672,iiduniA4N50,50,7
8,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.397117,8.534240,9.677539,10.824344,11.973087,13.122930,14.273423,iiduniA4N50,50,8
9,-37.077630,-35.926315,-34.775031,-33.623757,-32.472460,-31.321144,-30.169861,-29.018587,-27.867290,-26.715974,...,7.401795,8.536894,9.679038,10.825189,11.973563,13.123198,14.273574,iiduniA4N50,50,9


In [15]:
renyi_hats_long = pd.melt(renyi_hats, id_vars=['sampname', 'nobs', 'values.sample'], 
                          var_name='sig2', value_name='renyi2')
renyi_hats_long.set_index(['sampname', 'nobs', 'values.sample'])

sig2     renyi2
sampname     nobs values.sample                  
iiduniA4N50  50   0                0.0 -37.077630
                  1                0.0 -37.077630
                  2                0.0 -37.077630
                  3                0.0 -37.077630
                  4                0.0 -37.077630
...                                ...        ...
iiduniA4N100 100  15             100.0  14.273583
                  16             100.0  14.273551
                  17             100.0  14.273553
                  18             100.0  14.273576
                  19             100.0  14.273600

[1960 rows x 2 columns]

In [ ]:
dfestswide = pd.merge(theta_hats, renyi_hats, how='outer', on=['sampname', 'nobs', 'values.sample'])
dfestswide

In [ ]:
dfests = pd.wide_to_long(dfestswide, 'kernelvar', i=['nobs', 'values.sample'], j='sig2', sep='_')
dfests
#print(dfests.columns)

In [ ]:
dfindex = dfests.index
dfindex

In [ ]:
print(list(zip(('apen', 'sampen', 'renyi'), theta_iiduniform(a))))
print(thetas)

In [ ]:
#dftheta = pd.json_normalize(data=thetas)
dftheta = pd.DataFrame(thetas, index=dfindex)
dftheta.rename(columns={'apen': 'theta.apen', 'sampen': 'theta.sampen', 'renyi': 'theta.renyi'}, inplace=True)
dftheta

In [ ]:
df = pd.concat([dfests, dftheta], axis=1)
df[df['nobs']== 100]

In [ ]:
dfests = pd.json_normalize(data=estimates, record_path=['values', 'theta_hats'], 
                       meta=["sampname", "nobs", ["values", "m"]])

meta=["sampname", "nobs", ["values", "sample"]

```{bibliography}
:filter: docname in docnames
```

In [5]:
def load_simulated(filename):
    with open(filename, 'r+') as fhand:
        data = json.load(fhand)
    return data

In [6]:
simulated_path = pathlib.Path(r"simulation_output\simulation_output\simulated_2022-08-22\iiduniA4Nsim5_2022-08-22T221528.json")
simulated = load_simulated(simulated_path)

In [7]:
samples = simulated['Simulated Samples']
bgstates = simulated['BitGenerator states']

In [9]:
n200 = samples['iiduniA4N200']
n1000 = samples['iiduniA4N1000']
n5000 = samples['iiduniA4N5000']
n7500 = samples['iiduniA4N7500']
n10000 = samples['iiduniA4N10000']
n20000 = samples['iiduniA4N20000']

In [10]:
n200 = np.array(n200)
n1000 = np.array(n1000)
n5000 = np.array(n5000)
n7500 = np.array(n7500)
n10000 = np.array(n10000)
n20000 = np.array(n20000)

In [12]:
print("200", n200.shape)
print("1000", n1000.shape)
print("5000", n5000.shape)
print("7500", n7500.shape)
print("10000", n10000.shape)
print("20000", n20000.shape)

200 (5, 200)
1000 (5, 1000)
5000 (5, 5000)
7500 (5, 7500)
10000 (5, 10000)
20000 (5, 20000)


In [14]:
n200

array([['b', 'a', 'a', 'c', 'a', 'c', 'a', 'b', 'd', 'a', 'a', 'd', 'c',
        'c', 'a', 'd', 'b', 'b', 'a', 'c', 'b', 'a', 'd', 'd', 'd', 'c',
        'b', 'a', 'd', 'a', 'c', 'd', 'd', 'b', 'b', 'c', 'c', 'b', 'a',
        'd', 'c', 'c', 'c', 'b', 'c', 'c', 'c', 'b', 'b', 'd', 'd', 'c',
        'c', 'a', 'b', 'b', 'd', 'd', 'a', 'b', 'b', 'd', 'b', 'c', 'a',
        'd', 'a', 'b', 'd', 'b', 'd', 'b', 'c', 'a', 'a', 'd', 'b', 'a',
        'c', 'c', 'd', 'c', 'a', 'b', 'c', 'c', 'a', 'a', 'd', 'c', 'c',
        'b', 'd', 'd', 'b', 'd', 'd', 'a', 'd', 'c', 'c', 'd', 'b', 'b',
        'd', 'c', 'a', 'd', 'b', 'b', 'c', 'b', 'b', 'b', 'd', 'b', 'c',
        'd', 'a', 'a', 'a', 'a', 'b', 'c', 'c', 'd', 'c', 'c', 'c', 'b',
        'b', 'a', 'a', 'a', 'a', 'c', 'd', 'a', 'b', 'a', 'b', 'a', 'd',
        'b', 'b', 'd', 'c', 'a', 'd', 'a', 'b', 'a', 'c', 'a', 'a', 'b',
        'c', 'a', 'a', 'd', 'a', 'c', 'a', 'a', 'd', 'c', 'd', 'd', 'd',
        'c', 'b', 'c', 'c', 'c', 'c', 'c', 'b', 'c'

In [16]:
np.array_equal(n200, n1000[:, 0:200])
n1000[0, 0:200]

array(['d', 'b', 'b', 'b', 'd', 'c', 'a', 'b', 'c', 'b', 'd', 'd', 'b',
       'a', 'd', 'c', 'a', 'c', 'd', 'a', 'a', 'b', 'd', 'd', 'd', 'b',
       'd', 'd', 'b', 'c', 'd', 'b', 'a', 'd', 'a', 'b', 'c', 'b', 'd',
       'c', 'd', 'a', 'c', 'c', 'b', 'd', 'c', 'b', 'c', 'd', 'c', 'b',
       'd', 'c', 'b', 'b', 'c', 'b', 'a', 'a', 'b', 'c', 'c', 'd', 'd',
       'a', 'c', 'a', 'a', 'b', 'c', 'c', 'c', 'c', 'd', 'a', 'b', 'a',
       'd', 'a', 'a', 'a', 'c', 'c', 'd', 'b', 'a', 'c', 'c', 'c', 'd',
       'a', 'd', 'b', 'c', 'c', 'c', 'c', 'c', 'a', 'a', 'c', 'd', 'a',
       'b', 'c', 'b', 'a', 'a', 'd', 'c', 'b', 'd', 'a', 'd', 'b', 'c',
       'a', 'c', 'c', 'a', 'a', 'd', 'a', 'c', 'a', 'b', 'c', 'b', 'c',
       'b', 'c', 'a', 'c', 'b', 'c', 'b', 'a', 'd', 'd', 'd', 'b', 'd',
       'a', 'c', 'c', 'b', 'a', 'd', 'a', 'a', 'a', 'a', 'b', 'a', 'd',
       'c', 'a', 'a', 'c', 'b', 'a', 'c', 'd', 'b', 'd', 'a', 'd', 'c',
       'a', 'c', 'a', 'd', 'b', 'a', 'c', 'c', 'd', 'b', 'b', 'd

197534829086275859791696459812082112370

In [30]:
bgstates

{'iiduniA4N200': {'initial state': {'bit_generator': 'PCG64',
   'state': {'state': 283615424150945336485605884985051301252,
    'inc': 74160500731266736194320534480258873429},
   'has_uint32': 0,
   'uinteger': 0},
  '0': {'bit_generator': 'PCG64',
   'state': {'state': 197534829086275859791696459812082112370,
    'inc': 74160500731266736194320534480258873429},
   'has_uint32': 0,
   'uinteger': 663597282},
  '1': {'bit_generator': 'PCG64',
   'state': {'state': 203972238839762233473166077445431121058,
    'inc': 74160500731266736194320534480258873429},
   'has_uint32': 0,
   'uinteger': 4090514080},
  '2': {'bit_generator': 'PCG64',
   'state': {'state': 282526071284123077236071814865232561618,
    'inc': 74160500731266736194320534480258873429},
   'has_uint32': 0,
   'uinteger': 2598118357},
  '3': {'bit_generator': 'PCG64',
   'state': {'state': 287192323698105632153822238099330427650,
    'inc': 74160500731266736194320534480258873429},
   'has_uint32': 0,
   'uinteger': 3347402723

In [16]:
print("Sample names in data", samples.keys())
print("Number of sample names", len(samples.keys()))
for k,v in samples.items():
    print(f"Number of samples for sample {k}", len(v))
print("Number of items BitGenerator states", len(bgstates.keys()))
print("Names of items in BitGenerator states", bgstates.keys())
for sampname in bgstates.keys():
    print(f"BitGenerator states for sample {sampname}:")
    for statedict in bgstates[sampname].keys():
        print(f"State index: {statedict}\tBitGenerator State: ", bgstates[sampname][statedict]['state'])

Sample names in data dict_keys(['iiduniA4N200', 'iiduniA4N1000', 'iiduniA4N5000', 'iiduniA4N7500', 'iiduniA4N10000', 'iiduniA4N20000'])
Number of sample names 6
Number of samples for sample iiduniA4N200 5
Number of samples for sample iiduniA4N1000 5
Number of samples for sample iiduniA4N5000 5
Number of samples for sample iiduniA4N7500 5
Number of samples for sample iiduniA4N10000 5
Number of samples for sample iiduniA4N20000 5
Number of items BitGenerator states 6
Names of items in BitGenerator states dict_keys(['iiduniA4N200', 'iiduniA4N1000', 'iiduniA4N5000', 'iiduniA4N7500', 'iiduniA4N10000', 'iiduniA4N20000'])
BitGenerator states for sample iiduniA4N200:
State index: initial state	BitGenerator State:  {'state': 283615424150945336485605884985051301252, 'inc': 74160500731266736194320534480258873429}
State index: 0	BitGenerator State:  {'state': 197534829086275859791696459812082112370, 'inc': 74160500731266736194320534480258873429}
State index: 1	BitGenerator State:  {'state': 203972

In [4]:
sim = Simulator(genRandseq, 2, seed=None)
print(sim.rng.bit_generator.state)
print(sim.bgstateseq)
samples = sim(['a', 'b', 'c'], 50)
print(sim.rng.bit_generator.state)
print(sim.bgstateseq)
samples = sim(['a', 'b', 'c'], 100)
print(sim.rng.bit_generator.state)
print(sim.bgstateseq)

{'bit_generator': 'PCG64', 'state': {'state': 75058408964912748410493062550065597388, 'inc': 144522040233871519577923611202216540145}, 'has_uint32': 0, 'uinteger': 0}
{'initial state': {'bit_generator': 'PCG64', 'state': {'state': 75058408964912748410493062550065597388, 'inc': 144522040233871519577923611202216540145}, 'has_uint32': 0, 'uinteger': 0}}
{'bit_generator': 'PCG64', 'state': {'state': 318621496402524747890246474972659535394, 'inc': 144522040233871519577923611202216540145}, 'has_uint32': 0, 'uinteger': 982196600}
{'initial state': {'bit_generator': 'PCG64', 'state': {'state': 75058408964912748410493062550065597388, 'inc': 144522040233871519577923611202216540145}, 'has_uint32': 0, 'uinteger': 0}, 0: {'bit_generator': 'PCG64', 'state': {'state': 75058408964912748410493062550065597388, 'inc': 144522040233871519577923611202216540145}, 'has_uint32': 0, 'uinteger': 0}, 1: {'bit_generator': 'PCG64', 'state': {'state': 320574916760946357868330425410625415925, 'inc': 14452204023387151

In [5]:
sim = Simulator(genRandseq, 2, seed=None)
print(sim.rng.bit_generator.state)
print(sim.bgstateseq)
obs = [50, 100]
for n in obs:
    samples = sim(['a', 'b', 'c'], n)
    #print(sim.rng.bit_generator.state)
    print(sim.bgstateseq)

{'bit_generator': 'PCG64', 'state': {'state': 209754956547866010846383328550427064023, 'inc': 166743912981930103388829921474337229591}, 'has_uint32': 0, 'uinteger': 0}
{'initial state': {'bit_generator': 'PCG64', 'state': {'state': 209754956547866010846383328550427064023, 'inc': 166743912981930103388829921474337229591}, 'has_uint32': 0, 'uinteger': 0}}
{'initial state': {'bit_generator': 'PCG64', 'state': {'state': 209754956547866010846383328550427064023, 'inc': 166743912981930103388829921474337229591}, 'has_uint32': 0, 'uinteger': 0}, 0: {'bit_generator': 'PCG64', 'state': {'state': 209754956547866010846383328550427064023, 'inc': 166743912981930103388829921474337229591}, 'has_uint32': 0, 'uinteger': 0}, 1: {'bit_generator': 'PCG64', 'state': {'state': 23015717056148564473411509476467810978, 'inc': 166743912981930103388829921474337229591}, 'has_uint32': 0, 'uinteger': 2660574354}, 'end': {'bit_generator': 'PCG64', 'state': {'state': 32697333364390506168940893095809102265, 'inc': 166743